Please make sure you have imported the necessary files from the 'Data Sets' folder within the Zip file.

**IMPORTANT** To have the code run correctly, please create a folder named 'coinDataSets' and add all the csv files from the respective folder inside the Zip. At the end of the video we submit, I demonstrate this.

In [6]:
!pip install fredapi #need this to use FRED API
!pip install pandas_datareader
!pip install wordcloud
!pip install yfinance
!pip install pandas
!pip install matplotlib
!pip install requests
!pip install beautifulsoup4
!pip install newspaper3k

In [13]:
def nick_q1 () :
  import pandas as pd
  import matplotlib.pyplot as plt

  # Import files and save as a pandas dataframe
  def create_dataframe(file_path):
      df = pd.read_csv(file_path)
      # Drop columns with no data
      df = df.dropna(axis=1, how='all')
      return df

  # Combine relevant data from two seperate dataframes
  def merge_dataframes(df1,df3):
      # Parse dates in the correct format for both dataframes
      df1['Date'] = pd.to_datetime(df1['Date'], format='%Y-%m-%d')
      df3['date'] = pd.to_datetime(df3['date'], format='%d-%b-%y')

      # Extract years from date columns in df1 and df3
      df1['Year'] = df1['Date'].dt.year
      df3['Year'] = df3['date'].dt.year

      # Find common years between df1 and df3
      common_years = set(df1['Year']).intersection(df3['Year'])

      # Filter dataframes to include only rows from common years
      df1_common = df1[df1['Year'].isin(common_years)]
      df3_common = df3[df3['Year'].isin(common_years)]

      # Merge the dataframes based on common years
      merged_df = pd.merge(df1_common, df3_common, on='Year', suffixes=('_df1', '_df3'))

      return merged_df

  # Create a line plot that compares the two data sets.
  def plot_data(merged_df):
      # Create a figure and axis
      plt.figure(figsize=(10, 6))
      ax = plt.gca()

      # Plot 'close' data
      merged_df.plot(kind='line', x='date', y='close', ax=ax, label='Closing Price', color='blue')

      # Plot 'SP500' data
      merged_df.plot(kind='line', x='Date', y='SP500', ax=ax, label='SP500 Index', color='red')

      # Set labels and title
      plt.xlabel('Date')
      plt.ylabel('S&P 500 Index')
      plt.title('Comparison of Closing Price and S&P 500 Index From 1986-2018')

      # Show legend
      plt.legend()

      # Display the plot
      plt.show()

  # Create variable to hold names of files to import
  file1 = 'sp500_index_data.csv'
  file3 = 'spx.csv'

  # Run create_dataframe funciton to import all files and store as dataframes
  df1 = create_dataframe(file1)
  df3 = create_dataframe(file3)

  # Run merge_dataframes to combine only elements whose dates were in both datasets into a single dataframe
  merged_df = merge_dataframes(df1, df3)

  # Run plot_data to create and display the line plot comparing the datasets
  plot_data(merged_df)

def nick_q2() :
  import pandas as pd
  import matplotlib.pyplot as plt
  from matplotlib.lines import Line2D

  # Import file and save as a pandas dataframe
  def create_dataframe(file_path):
      df = pd.read_csv(file_path)
      # Drop columns with no data
      df = df.dropna(axis=1, how='all')
      return df

  # Clean up the dataframe to put the data in a more usable state
  def clean_dataframes(df1):
      # Parse dates in the correct format
      df1['Date'] = pd.to_datetime(df1['Date'], format='%Y-%m-%d')

      # Extract years from date column in df1
      df1['Year'] = df1['Date'].dt.year

      # Remove all data prior to 1940
      df1 = df1[df1['Year'] >= 1940]

      return df1

  # Calculate the Market Phase (Bear, Bull, or Neutral), based primarily off of the +/-20% threshold
  def calculate_market_phase(clean_df):
      # Set 'Date' column as index
      clean_df.set_index('Date', inplace=True)

      # Resample data on a monthly basis and calculate monthly percentage changes
      monthly_df = clean_df.resample('2M').last()
      monthly_df['Monthly_Return'] = monthly_df['SP500'].pct_change() * 100

      # Initialize variables for consecutive changes
      c_bear = []
      c_bull = []
      bear_total = 0
      bull_total = 0

      # Determine market phase for each month
      monthly_df['Market_Phase'] = 'Neutral'

      for index, row in monthly_df.iterrows():

          if row[10] > 0:
              c_bull.append(index)
              bull_total = bull_total + row[10]
              if len(c_bear) > 1:
                  if bear_total <= -20:
                      for date in c_bear:
                          monthly_df.at[date, 'Market_Phase'] = 'Bear'
                  c_bear = []
                  bear_total = 0
          elif row[10] < 0:
              c_bear.append(index)
              bear_total = bear_total + row[10]
              if len(c_bull) > 1:
                  if bull_total >= 20:
                      for date in c_bull:
                          monthly_df.at[date, 'Market_Phase'] = 'Bull'
                  c_bull = []
                  bull_total = 0

      # Extract the Market_Phase column into its own list to simplify element alterations
      col_market_phase = monthly_df['Market_Phase'].tolist()

      # Improve data for plotting by determining which Neutral phases prematurly interupted or ended a Bear or Bull Phase
      c_neutral = 0
      previous_phase = None
      for i in range(len(col_market_phase)):
          if col_market_phase[i] == 'Neutral':
              c_neutral = c_neutral + 1
          else:
              if c_neutral > 0 and c_neutral < 10 and previous_phase == col_market_phase[i]:
                  col_market_phase[i - c_neutral:i] = [previous_phase] * c_neutral
              c_neutral = 0
              previous_phase = col_market_phase[i]

      # Send the list of market phase elements back to the Market_Phase dataframe column
      monthly_df['Market_Phase'] = col_market_phase

      # Iterate through dataframe to further weed out any premature phase terminations using a lower threshold to catch specific mistakes
      for index, row in monthly_df.iterrows():
          if row['Monthly_Return'] > 5 and row['Market_Phase'] == 'Bear':
              monthly_df.at[index, 'Market_Phase'] = 'Neutral'
          elif row['Monthly_Return'] < -5 and row['Market_Phase'] == 'Bull':
              monthly_df.at[index, 'Market_Phase'] = 'Neutral'

      return monthly_df

  # Create a plot that displays the S&P 500 index and identifies each phase with a different color
  def plot_data(result_df):

      # Create a color mapt for determing which color to make each segment based on phase
      color_map = {'Bull': 'green', 'Neutral': 'blue','Bear': 'red'}

      # Create phase label dictionary to use in legend creation
      phase_labels = {'Bear': 'Bear Market','Neutral': 'Neutral Market','Bull': 'Bull Market'}


      # Create a line chart with a colored segmented line based on market phase
      plt.figure(figsize=(10, 6))

      # Initialize the current phase and segment as None
      current_phase = None
      current_segment = None

      # Iterate through dataframe and identify where each phase ends so that the line can be terminated and a new line with the proper color can be stitched on
      for index, row in result_df.iterrows():
          if current_phase is None:
              current_phase = row[11]
              current_segment = index
          elif current_phase != row[11]:
              plt.plot(result_df.loc[current_segment:index].index, result_df.loc[current_segment:index]['SP500'], linestyle='-', color=color_map[current_phase])
              current_phase = row[11]
              current_segment = index

      # Plot the last segment outside of the for loop to terminate the plot
      if current_phase is not None:
          plt.plot(result_df.loc[current_segment:].index, result_df.loc[current_segment:]['SP500'], linestyle='-', color=color_map[current_phase])


      # Set parameters for creating the plot to make it look better and properly convey data
      plt.title('Change in Market Phase in Relation to S&P 500 Index from 1940 to 2018')
      plt.xlabel('Year')
      plt.ylabel('S&P 500 Index')
      plt.grid(False)
      plt.xticks(rotation=45)
      plt.tight_layout()

      # Create a legend because the built in one did not like the fact that I technically have like a million lines
      legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=phase_labels[phase]) for phase, color in color_map.items()]
      plt.legend(handles=legend_elements, loc='upper left', fontsize=12)


      # Display the plot
      plt.show()


  # Create variable to hold names of files to import
  file1 = 'sp500_index_data.csv'

  # Run create_dataframe funciton to import all files and store as dataframes
  df1 = create_dataframe(file1)

  # Run clean_df to organize the dataframe in a way that it can be used to calculate market phases
  clean_df = clean_dataframes(df1)

  # Run calculate_market_phase to calculate the % change in S&P 500 index which can be used to roughly determine market phase
  result_df = calculate_market_phase(clean_df)

  # Create a line plot to most effectively display how the market phase changes correlate with changes in the S&P 500 index over time
  plot_data(result_df)

def nick_q3() :
  import requests
  import pandas as pd
  import matplotlib.pyplot as plt
  from matplotlib.lines import Line2D

  # FRED API endpoint for the VIX index
  api_url = 'https://api.stlouisfed.org/fred/series/observations'

  # API key
  api_key = '77b065d6da18e67b78973664ea28508d'

  # Import file and save as a pandas dataframe
  def create_dataframe(file_path):
      df = pd.read_csv(file_path)
      # Drop columns with no data
      df = df.dropna(axis=1, how='all')
      return df

  # Clean up the dataframe to put the data in a more usable state
  def clean_dataframes(df1):
      # Parse dates in the correct format
      df1['Date'] = pd.to_datetime(df1['Date'], format='%Y-%m-%d')

      # Extract years from date column in df1
      df1['Year'] = df1['Date'].dt.year

      # Remove all data prior to 1940
      df1 = df1[df1['Year'] >= 1940]

      return df1

  # Calculate the Market Phase (Bear, Bull, or Neutral), based primarily off of the +/-20% threshold
  def calculate_market_phase(clean_df):
      # Set 'Date' column as index
      clean_df.set_index('Date', inplace=True)

      # Resample data on a monthly basis and calculate monthly percentage changes
      monthly_df = clean_df.resample('2M').last()
      monthly_df['Monthly_Return'] = monthly_df['SP500'].pct_change() * 100

      # Initialize variables for consecutive changes
      c_bear = []
      c_bull = []
      bear_total = 0
      bull_total = 0

      # Determine market phase for each month
      monthly_df['Market_Phase'] = 'Neutral'

      for index, row in monthly_df.iterrows():

          if row[10] > 0:
              c_bull.append(index)
              bull_total = bull_total + row[10]
              if len(c_bear) > 1:
                  if bear_total <= -20:
                      for date in c_bear:
                          monthly_df.at[date, 'Market_Phase'] = 'Bear'
                  c_bear = []
                  bear_total = 0
          elif row[10] < 0:
              c_bear.append(index)
              bear_total = bear_total + row[10]
              if len(c_bull) > 1:
                  if bull_total >= 20:
                      for date in c_bull:
                          monthly_df.at[date, 'Market_Phase'] = 'Bull'
                  c_bull = []
                  bull_total = 0

      # Extract the Market_Phase column into its own list to simplify element alterations
      col_market_phase = monthly_df['Market_Phase'].tolist()

      # Improve data for plotting by determining which Neutral phases prematurly interupted or ended a Bear or Bull Phase
      c_neutral = 0
      previous_phase = None
      for i in range(len(col_market_phase)):
          if col_market_phase[i] == 'Neutral':
              c_neutral = c_neutral + 1
          else:
              if c_neutral > 0 and c_neutral < 10 and previous_phase == col_market_phase[i]:
                  col_market_phase[i - c_neutral:i] = [previous_phase] * c_neutral
              c_neutral = 0
              previous_phase = col_market_phase[i]

      # Send the list of market phase elements back to the Market_Phase dataframe column
      monthly_df['Market_Phase'] = col_market_phase

      # Iterate through dataframe to further weed out any premature phase terminations using a lower threshold to catch specific mistakes
      for index, row in monthly_df.iterrows():
          if row['Monthly_Return'] > 5 and row['Market_Phase'] == 'Bear':
              monthly_df.at[index, 'Market_Phase'] = 'Neutral'
          elif row['Monthly_Return'] < -5 and row['Market_Phase'] == 'Bull':
              monthly_df.at[index, 'Market_Phase'] = 'Neutral'

      return monthly_df

  # Utilize the FRED API to retrieve the historical CBOE Volatility data from the Federal Reserve
  def retrieve_VIX(api_key, api_url):
      # Parameters for the API request
      params = {

          'series_id': 'VIXCLS',
          'api_key': api_key,
          'file_type': 'json',
          'observation_start': '1990-01-02',
          'observation_end': '2018-12-31'
          }

      # Use a try-except block to catch any errors that may occur while fetching data
      try:
          response = requests.get(api_url, params=params)

          if response.status_code == 200:
              # Save successful query response using json processing
              data = response.json()

              # Extract the observations from the federal reserve
              observations = data['observations']

              # Create a list of dictionaries for DataFrame creation
              vix_data = [{'date': obs['date'], 'vix_value': obs['value']} for obs in observations]

              # Create a DataFrame from the list of dictionaries
              hist_vix = pd.DataFrame(vix_data)
          else:
              # If the query fails print appropriate response code
              print(f'Request failed with status code: {response.status_code}')

      # If an Exception is thrown print the error that occured
      except Exception as error:
          print(f'An error occurred: {error}')
      return hist_vix

  # Add the market phase data appropriate for the date ranges in the vix_data dataframe
  def combine_vix_snp500(vix_data, result_df):

      # Filter data down to only the years that both datasets share
      result_df = result_df[result_df.index.year >= 1990]

      # Extract market phase data into seperate list to process before transfering to vix_data
      col_market_phase = result_df['Market_Phase'].tolist()
      vix_data = vix_data.assign(Market_Phase='')
      vix_data.set_index('date', inplace=True)

      # Convert the index to a DatetimeIndex
      vix_data.index = pd.to_datetime(vix_data.index)

      # Remove null data points that were represented with '.'
      vix_data = vix_data[vix_data['vix_value'] != '.']

      # Convert vix value data to float variable type so that an average can be calculated for each month
      vix_data['vix_value'] = vix_data['vix_value'].astype(float)

      # Group by month and calculate the average vix_value for each month
      monthly_avg_vix = vix_data.groupby(pd.Grouper(freq='2M'))['vix_value'].mean()

      # Reset index to get a DataFrame with where the date is a column
      vix_data = monthly_avg_vix.reset_index()

      # Remove the last 4 entries from the list because I couldn't get the objects to be the same length
      vix_data = vix_data.drop(vix_data.tail(4).index)

      # Pass the market phase data into the vix dataframe
      vix_data['Market_Phase'] = col_market_phase

      # Make sure the date is in the proper format again
      vix_data['date'] = pd.to_datetime(vix_data['date'])

      return vix_data

  # Plot the data using the calcualted average CBOE Volatility data from the Federal Reserve and the market phases calculated using the S&P 500 index
  def plot_data(merged_df):
      color_map = {'Bull': 'green', 'Neutral': 'blue','Bear': 'red'}
      phase_labels = {'Bear': 'Bear Market','Neutral': 'Neutral Market','Bull': 'Bull Market'}

      # Create a line chart of VIX data with segmented line for each market phase calculated from S&P500 data
      plt.figure(figsize=(10, 6))

      # Initialize the current phase and segment as None
      current_phase = None
      current_segment = None

      # Iterate through dataframe and identify where each phase ends so that the line can be terminated and a new line with the proper color can be stitched on
      for index, row in merged_df.iterrows():
          if current_phase is None:
              current_phase = row[2]
              current_segment = index
          elif current_phase != row[2]:
              plt.plot(merged_df.loc[current_segment:index]['date'], merged_df.loc[current_segment:index]['vix_value'], linestyle='-', color=color_map[current_phase])
              current_phase = row[2]
              current_segment = index

      # Plot the last segment
      if current_phase is not None:
          plt.plot(merged_df.loc[current_segment:]['date'], merged_df.loc[current_segment:]['vix_value'], linestyle='-', color=color_map[current_phase])

      # Set parameters for creating the plot to make it look better and properly convey data
      plt.title('CBOE Volatility Index and Market Phase Comparison from 1990 to 2018')
      plt.xlabel('Year')
      plt.ylabel('CBOE Volatility Index')
      plt.grid(False)
      plt.xticks(rotation=45)
      plt.tight_layout()

      # Create a legend because the built in one did not like the fact that I technically have like a million lines
      legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=phase_labels[phase]) for phase, color in color_map.items()]
      plt.legend(handles=legend_elements, loc='upper left', fontsize=12)

      # Display the plot
      plt.show()

  # Create variable to hold names of file to import
  file1 = 'sp500_index_data.csv'

  # Run create_dataframe funciton to import all files and store as dataframes
  df1 = create_dataframe(file1)

  # Run clean_df to organize the dataframe in a way that it can be used to calculate market phases
  clean_df = clean_dataframes(df1)

  # Run calculate_market_phase to calculate the % change in S&P 500 index which can be used to roughly determine market phase
  result_df = calculate_market_phase(clean_df)

  # Utilize FRED api to retrieve CBOB data from federal reserve and parse into a dataframe
  vix_data = retrieve_VIX(api_key, api_url)

  # Filter out unrelevant data and combine into a single file for plotting
  merged_df = combine_vix_snp500(vix_data, result_df)

  # Create a line plot to most effectively display how the market phase changes correlate with the CBOE Volatility data
  plot_data(merged_df)




def noah_q1 () :
  #import necessary libraries
  import pandas as pd
  import regex as re
  import matplotlib.pyplot as plt
  import seaborn as sns
  import numpy as np

  #extract SP500 prices
  def convert_lists_dict(averageS500_list, averageCPI_list) :
    convertSP500 = list()
    convertCPI = list()

    for value in averageS500_list:
      convertSP500.append(value[1])

    for value in averageCPI_list:
      convertCPI.append(value[1])

    years.append(2017)

    return convertSP500, convertCPI

  #percent of change in average S&P price from year before
  def percents_S500() :
    counter = 0
    percents_S500 = list()
    years = list()
    for index in averageS500_list :
      try :
        counter += 1
        hold_before_S500 = averageS500_list[counter - 1] #holds value before true index placement
        hold_past_S500 = averageS500_list[counter] #holds value of true index placement

        percent_diff_S500 = (hold_past_S500[1] / hold_before_S500[1]) - 1     #calculates % change in price from year before

        percents_S500.append(percent_diff_S500)
        years.append(int(hold_past_S500[0]))

      except IndexError :
        continue

    return percents_S500, years

  #percent of change in average CPI from year before
  def percents_CPI() :
    counter = 0
    percents_CPI = list()
    for index in averageCPI_list :
      try :
        counter += 1
        hold_before_CPI = averageCPI_list[counter - 1]  #same structure as SP500 calculation of % change
        hold_past_CPI = averageCPI_list[counter]

        percent_diff_CPI = (hold_past_CPI[1] / hold_before_CPI[1]) - 1

        percents_CPI.append(percent_diff_CPI)

      except IndexError :
        continue

    return percents_CPI




  #load data into pandas
  data_df = pd.read_csv('sp500_index_data.csv')

  #gather the average change in CPI and average change in S&P
  years = len(data_df) / 12 #amount of years included in the df (CPI and SP500 have same length)

  averageCPI_list = list()
  averageS500_list = list()

  x = 0  #our index holder
  is_twelve = 1
  total_CPI = 0
  total_S500 = 0
  for line in data_df.index:
    if x == 1764 : #once we get to the most recent complete year, break
      break


    total_CPI += data_df['Consumer Price Index'][x]   #running totals for the years
    total_S500 += data_df['SP500'][x]

    if is_twelve == 12 :
      is_twelve = 0
      average_CPI = total_CPI / 12      #find averages for the years
      average_S500 = total_S500 / 12

      year = data_df['Date'][x]
      extraction = r'\b\d{4}\b'     #extract the year
      year = re.search(extraction, year).group()


      averageCPI_list.append([year, average_CPI])    #storing list of lists with year : values as key-value pairs
      averageS500_list.append([year,average_S500])

      total_CPI = 0
      total_S500 = 0
      average_CPI = 0
      average_S500 = 0
      year = ''


    x += 1  #holds our index placement
    is_twelve += 1  #tracks when we hit 12 months in the dataset

  percents_SP500, years = percents_S500()
  percents_CPI = percents_CPI()

  start = len(years)
  start -= 50   #only access past 50 years
  percents_dict = {
      'Years' : years[start:],
      'SP500' : percents_SP500[start:],
      'CPI' : percents_CPI[start:]
  }
  dataframe_yearly_percent = pd.DataFrame.from_dict(percents_dict)


  start = len(years)
  start -= 50
  SP500_values, CPI_values = convert_lists_dict(averageS500_list, averageCPI_list)
  averages_dict = {
      'Years' : years[start:],
      "SP500" : SP500_values[start:],
      "CPI" : CPI_values[start:]

  }

  dataframe_yearly_avg = pd.DataFrame.from_dict(averages_dict)


  #PLOTTING
  plt.figure(figsize = (20,5))

  plt.subplot(1,2,1)
  correlation_percent = dataframe_yearly_percent['CPI'].corr(dataframe_yearly_percent['SP500'])    #find correlation between values, add to title
  sns.regplot(data = dataframe_yearly_percent, x = 'CPI', y = 'SP500').set(title=f'Correlation Between CPI & S&P500 Yearly % Change - (Corr. {correlation_percent:.3f})')

  plt.subplot(1,2,2)
  correlation_avg = dataframe_yearly_avg['CPI'].corr(dataframe_yearly_avg['SP500'])
  sns.regplot(data = dataframe_yearly_avg, x = 'CPI', y = 'SP500').set(title=f'Correlation Between CPI & S&P500 Yearly Average Value - (Corr. {correlation_avg:.3f})')



  #plot twin y-axis graph with shared x-axis. Helps visualize relationship over time.
  ax = dataframe_yearly_percent.plot(x='Years', y='SP500', legend=False)
  ax2 = ax.twinx()
  dataframe_yearly_percent.plot(x='Years', y='CPI', ax=ax2, legend=False, color='r')
  ax.figure.legend()
  ax.set_ylabel('SP500')
  ax2.set_ylabel('CPI')
  plt.title('Percent Change in S&P500 and CPI Over Time')
  plt.show()

  ax = dataframe_yearly_avg.plot(x='Years', y='SP500', legend=False)
  ax2 = ax.twinx()
  dataframe_yearly_avg.plot(x='Years', y='CPI', ax=ax2, legend=False, color='r')
  ax.figure.legend()
  ax.set_ylabel('SP500')
  ax2.set_ylabel('CPI')
  plt.title('S&P500 and CPI Over Time')
  plt.show()


def noah_q2() :
  from fredapi import Fred
  fred = Fred(api_key='cb46a20f480a8c2bc9649eb1e6b4b93d') #my key access from FRED API
  import pandas as pd
  import matplotlib.pyplot as plt


  def dataframes (GNP_dict, SP500_dict) :
    df_GNP = pd.DataFrame.from_dict(GNP_dict)     #build our new dataframes
    df_SP500 = pd.DataFrame.from_dict(SP500_dict)
    return df_GNP, df_SP500

  GNP = fred.get_series('GNP') #get data from FRED api as a series

  SP500 = fred.get_series('SP500')


  #take out values, convert to dataframe
  years_GNP = GNP.index
  start = len(years_GNP) - 40
  GNP_values = GNP.values

  years_SP500 = SP500.index
  daily_SP500 = SP500.values


  GNP_dict = {
      'Years_GNP' : years_GNP[start:], #first quarter of 2013 to last quarter of 2022. SP500 only has prices for last 10 years, so we match GNP timeline.
      'GNP' : GNP_values[start:],
  }
  SP500_dict = {
      'Years_SP500' : years_SP500,
      'SP500' : daily_SP500
  }

  df_GNP, df_SP500 = dataframes(GNP_dict, SP500_dict)

  #PLOTTING
  #plotting twin y-axis graph with shared x-acis to show relationship over time
  plt.figure(figsize = (20,5))

  ax = df_SP500.plot(x='Years_SP500', y='SP500', legend = False)
  ax2= ax.twinx()
  df_GNP.plot(x='Years_GNP', y='GNP', ax=ax2, legend=False, color='r')
  ax.figure.legend()
  ax2.set_ylabel('GNP (in thousands of billions)')
  ax.set_ylabel('S&P (in thousands)')
  ax.set_xlabel('Years')
  correlation_percent = df_SP500['SP500'].corr(df_GNP['GNP'])   #find correlation between GNP and SP500 over past decade
  plt.title(f'S&P500 and GNP Over Time (r: {correlation_percent})')
  plt.show()


def noah_q3() :
  from fredapi import Fred
  key = 'cb46a20f480a8c2bc9649eb1e6b4b93d' #my api key access for FRED API
  import pandas as pd
  import matplotlib.pyplot as plt
  import requests
  from bs4 import BeautifulSoup
  import urllib.request
  import regex as re
  import numpy as np
  start_date = '2013-01-01' #want data for past decade from FRED API
  end_date = '2023-08-08'

#get subset dataframe from SP500 JSON
  def get_subset (key, start_date, end_date) :
    seriesID = 'SP500'
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={seriesID}&api_key={key}&file_type=json" + \
          f"&observation_start={start_date}&observation_end={end_date}&units=lin"

    SP500 = requests.get(url)
    SP500_json = SP500.json()  #FRED will return SP500 data in JSON which we convert to a dataframe below

    # Convert JSON data to a DataFrame
    df = pd.DataFrame(SP500_json['observations'])

    # Rename columns for clarity
    df.rename(columns={'date': 'Date', 'value': 'Value'}, inplace=True)


    #re-arrange data in columnds for plotting purposes. We want dates and values in ascending order from L-R on the plot
    df['Date'] = df['Date'].iloc[::-1].reset_index(drop=True)
    df['Value'] = df['Value'].iloc[::-1].reset_index(drop=True)
    df_subset = df.iloc[::91, :] #creating a subset dataframe with every 91 value. 91 represents the amount of days per quarter in a year

    df_subset = df_subset.iloc[::-1].reset_index(drop=True)
    df_subset = df_subset.drop(22) #drop a value that was causing error. Value was a '.' instead of a SP500 price
    return df_subset


  def convert_SP500_float (df_subset) :
    #convert SP500 dataframe into legible float
    years_SP500 = list()
    date_list = df_subset['Date'].tolist()  #extract date column into list
    for value in date_list :
      pattern = r'^(\d{4})-\d{2}-\d{2}$' #extract years from the date format
      match = re.search(pattern, value)
      x = match.group(1)
      years_SP500.append(int(x))  #add extracted year into years_SP500 list

    values_SP500 = list()
    value_list = df_subset['Value'].tolist()  #extract SP500 prices into list and convert to float. Append to values_SP500 list
    for value in value_list :
      x = float(value)
      values_SP500.append(int(x))

    return years_SP500, values_SP500

  #webscrape for the data we need
  def webscrape (soup) :
    data = {} #holds prices of US Cruse Oil FPP with their decade. Data stored as key : list (decade : prices)

    for row in soup.find_all('tr'): #find all tr tags
        cells = row.find_all('td') #find rows with td tags
        if cells and len(cells) > 1:
            year = cells[0].text.strip() #extract year
            if year == "2010's" or year == "2020's": #only extract decades that can be used in correlation with SP500 timeline (past decade)
                try:
                    prices = [float(cell.text.strip()) if cell.text.strip() != '' else None for cell in cells[1:]] #extract prices with an exception if there is no value
                    data[year] = prices
                except ValueError:
                    pass
    return data


  #webscrape oil data from web
  url = 'https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=pet&s=f000000__3&f=a'
  response = requests.get(url)
  mystr = response.content #holds html
  soup = BeautifulSoup(mystr, 'html.parser') #use beautiful soup to parse html
  data = webscrape(soup)


  years = list()
  values = list()
  year_counter = 2010

  #create years to match prices that exist in data{}
  for value in data.values() :
    for item in value:
      years.append(year_counter)
      values.append(item)

      year_counter += 1

  #only use years from past decade (2013-2023)
  oil_dict = {
      'Years' : years[3:13],
      'US FPP' : values[3:13],

  }


  oil_df = pd.DataFrame.from_dict(oil_dict)
  df_subset = get_subset(key, start_date, end_date)
  years_SP500, values_SP500 = convert_SP500_float(df_subset)


  SP500_dict = {
      'Years' : years_SP500,
      'SP500 Prices' : values_SP500
  }


  df_SP500 = pd.DataFrame.from_dict(SP500_dict)

  #PLOTTING
  # Plot both dataframes on the same x-axis with a secondary y-axis
  plt.figure(figsize=(20,5))

  ax = oil_df.plot(x='Years',y='US FPP',legend=False)
  ax2=ax.twinx()
  df_SP500.plot(x='Years',y='SP500 Prices', ax=ax2, legend=False, color='r')
  ax.figure.legend()
  ax.set_ylabel('US Crude Oil First Purchase Prices')
  ax.set_xlabel('Years')
  ax2.set_ylabel('SP500 Prices')
  correlation_percent = df_SP500['SP500 Prices'].corr(oil_df['US FPP'])  #correlation between US Crude Oil FPP and SP500 prices over past decade
  plt.title(f'US Crude Oil FPP and S&P 500 Over Time\n(r: {correlation_percent:.3f})')
  plt.show()


def siya_q1() :
  #Importing for fetching financial data
  import yfinance as yf
  #Importing for data manipulation and analysis
  import pandas as pd
  #Importing for data visualization
  import matplotlib.pyplot as plt
  #Importing data module for economic data
  from pandas_datareader import data as pdr
  #Importing for creating word clouds
  from wordcloud import WordCloud

  #Analyzing S&P 500 Performance:
  #This program uses historical S&P 500 data and interest rates, then visualizes the performance of the S&P 500 index during different presidential administrations
  #The goal is to analyze how various presidential terms have influenced the S&P 500's historical performance and identify visible trends

  #FRED API Personal Key
  fred_api_key = '8dd835b14e8ddd845fad438c41e2009a'

  #Finding S&P 500 data using yfinance
  ticker = "^GSPC"
  start_date = "1957-01-01"
  end_date = "2023-01-01"
  #Fetch S&P 500 data within the specified date range
  spx_data = yf.download(ticker, start=start_date, end=end_date)

  #Finding historical interest rates data
  fred_data = pdr.DataReader("GS10", "fred", start=start_date, end=end_date, api_key=fred_api_key)
  #Extract the GS10 interest rate data
  historical_interest_rates_data = fred_data["GS10"]

  #Finding the years with the highest interest rates
  highest_rate_year = historical_interest_rates_data.idxmax().year

  #Finding the corresponding S&P 500 data for the year with the highest interest rate and filtering it
  spx_data_highest_rate_year = spx_data[spx_data.index.year == highest_rate_year]

  #Presidential administration periods, start and end years
  presidential_periods = [
      {"start": 1981, "end": 1988, "name": "Reagan"},
      {"start": 1989, "end": 1992, "name": "Bush Sr."},
      {"start": 1993, "end": 2000, "name": "Clinton"},
      {"start": 2001, "end": 2008, "name": "Bush Jr."},
      {"start": 2009, "end": 2016, "name": "Obama"},
      {"start": 2017, "end": 2024, "name": "Trump"},
      {"start": 2025, "end": 2032, "name": "Future President"}
  ]

  #Plotting the S&P 500's historical performance during presidential administration periods
  plt.figure(figsize=(10, 6))
  #Plotting the overall S&P 500 performance
  spx_data["Adj Close"].plot(label="S&P 500")

  for period in presidential_periods:
      start_year = period["start"]
      end_year = period["end"]
      period_data = spx_data[(spx_data.index.year >= start_year) & (spx_data.index.year <= end_year)]
      #Plotting S&P 500 performance during each presidential period
      period_data["Adj Close"].plot(label=period["name"])

  #Specifying plot's visual aspects
  spx_data_highest_rate_year["Adj Close"].plot(marker="o", linestyle="", markersize=8, color="red", label="Highest Rate Year")
  plt.title(f"S&P 500 Performance and Presidential Administrations")
  plt.xlabel("Date")
  plt.ylabel("Adjusted Close Price")
  plt.legend()
  #Printing the visual
  plt.show()

  #Word Cloud visualization for presidential administrations' impact on the S&P 500

  #Creating a text string containing the names of presidential administrations
  #This will be used to generate the word cloud
  presidential_impact_text = " ".join([f"{period['name']} administration" for period in presidential_periods])

  #Creating a WordCloud object with specified settings
  wordcloud = WordCloud(
      width=800, height=400, background_color='black', colormap='viridis',
      contour_color='white', contour_width=1, min_word_length=4,
      max_font_size=100, relative_scaling=0.5
  ).generate(presidential_impact_text)

  #Creating a new figure for the word cloud visualization
  plt.figure(figsize=(6, 4))

  #Displaying the word cloud image with bilinear interpolation
  plt.imshow(wordcloud, interpolation='bilinear')

  #Turning off axis labels and ticks
  plt.axis("off")

  #Setting the title for the word cloud visualization
  plt.title("Word Cloud: Presidential Administrations' Impact on S&P 500", fontsize=14)

  #Adjusting layout for better spacing
  plt.tight_layout()

  #Displaying the word cloud plot
  plt.show()



def siya_q2() :
  #Importing for fetching financial data
  import yfinance as yf
  #Importing for data manipulation and analysis
  import pandas as pd
  #Importing for data visualization
  import matplotlib.pyplot as plt
  #Importing data module for economic data
  from pandas_datareader import data as pdr
  #Importing for creating word clouds
  from wordcloud import WordCloud
  #Finding S&P 500 data using yfinance
  ticker = "^GSPC"
  start_date = "1957-01-01"
  end_date = "2023-01-01"
  spx_data = yf.download(ticker, start=start_date, end=end_date)

  #Calculating the yearly percentage change for the S&P 500
  spx_data["Yearly Change %"] = spx_data["Adj Close"].pct_change(periods=1) * 100

  #Finding the years with the worst yearly change percentage
  worst_years = spx_data[spx_data["Yearly Change %"].notnull()].nsmallest(5, "Yearly Change %").index.year

  #Plotting the S&P 500's historical performance and mark the worst years using matplotlib
  plt.figure(figsize=(10, 6))
  spx_data["Adj Close"].plot(label="S&P 500")
  plt.scatter(spx_data.loc[spx_data.index.year.isin(worst_years), :].index, spx_data.loc[spx_data.index.year.isin(worst_years), "Adj Close"], color="orange", marker="v", label="Worst Years")

  #Setting the title of the plot
  plt.title("S&P 500 Performance and Worst Years by Yearly Change %")
  #Setting the label for the x-axis
  plt.xlabel("Date")
  #Setting the label for the y-axis
  plt.ylabel("Adjusted Close Price")
  #Displaying the legend in the plot
  plt.legend()
  #Printing the visual
  plt.show()

  #Printing the list of historically worst years by yearly change percentage
  print("Historically worst years by yearly change percentage:")

  #Looping through each year in the list of worst years
  for year in worst_years:
      #Printing the year and its yearly change percentage
      print(f"{year}: {spx_data.loc[spx_data.index.year == year, 'Yearly Change %'].values[0]:.2f}%")
  print("\n")


def siya_q3() :
  #Importing for fetching financial data
  import yfinance as yf
  #Importing for data manipulation and analysis
  import pandas as pd
  #Importing for data visualization
  import matplotlib.pyplot as plt
  #Importing data module for economic data
  from pandas_datareader import data as pdr
  #Importing for creating word clouds
  from wordcloud import WordCloud
  #Finding S&P 500 data using yfinance

  #S&P 500 Performance: Best and Lowest Years
  #This code uses historical S&P 500 data and identifies the years with the highest and lowest values of the index.
  #It then visualizes the performance of the S&P 500 over time, highlighting the best and lowest years, and provides information on these years.

  #Finding S&P 500 data using yfinance
  ticker = "^GSPC"
  start_date = "1957-01-01"
  end_date = "2023-01-01"
  #S&P 500 data within the specified date range
  spx_data = yf.download(ticker, start=start_date, end=end_date)

  #Resampling the data to find the years with the highest and lowest values of the S&P 500
  spx_yearly = spx_data["Adj Close"].resample('Y').agg({"Adj Close": "last"})
  #Finding the year with the highest S&P 500 value
  best_year = spx_yearly["Adj Close"].idxmax().year
  #Finding the year with the lowest S&P 500 value
  lowest_year = spx_yearly["Adj Close"].idxmin().year

  #Plotting the S&P 500's historical performance and mark the best and lowest years
  plt.figure(figsize=(10, 6))
  spx_data["Adj Close"].plot(label="S&P 500")  # Plot the overall S&P 500 performance
  plt.scatter(pd.Timestamp(f'{best_year}-12-31'), spx_yearly.loc[pd.Timestamp(f'{best_year}-12-31'), "Adj Close"], color="green", marker="^", label="Best Year")  # Mark the best year with a green triangle marker
  plt.scatter(pd.Timestamp(f'{lowest_year}-12-31'), spx_yearly.loc[pd.Timestamp(f'{lowest_year}-12-31'), "Adj Close"], color="red", marker="v", label="Lowest Year")  # Mark the lowest year with a red triangle marker

  plt.title("S&P 500 Performance and Best/Lowest Years")
  plt.xlabel("Date")
  plt.ylabel("Adjusted Close Price")
  plt.legend()
  #Displaying the plot
  plt.show()

  #Printing information about the best and lowest years
  print(f"The best year for the S&P 500 was: {best_year}")
  print(f"The lowest year for the S&P 500 was: {lowest_year}")
  print("\n")




def brady_q1() :
  import yfinance as yf
  import pandas as pd
  import matplotlib.pyplot as plt
  from fredapi import Fred

  # Fetch S&P 500 data using yfinance
  ticker = "^GSPC"
  start_date = "1957-01-01"
  end_date = "2023-01-01"
  spx_data = yf.download(ticker, start=start_date, end=end_date)

  # Fetch historical interest rates data using the FRED API
  fred = Fred(api_key='767f7894f66f59b2796e04a5525ad91e')
  historical_interest_rates_data = fred.get_series("DGS10")

  # Find the years with the highest interest rates
  highest_rate_year = historical_interest_rates_data.idxmax().year

  # Find the corresponding S&P 500 data for the year with the highest interest rate
  spx_data_highest_rate_year = spx_data[spx_data.index.year == highest_rate_year]

  # Plot the S&P 500's historical performance during the year with the highest interest rate using matplotlib
  plt.figure(figsize=(10, 6))
  spx_data["Adj Close"].plot(label="S&P 500")
  spx_data_highest_rate_year["Adj Close"].plot(marker="o", linestyle="", markersize=8, color="red", label="Highest Rate Year")
  plt.title(f"S&P 500 Performance during {highest_rate_year} (Highest Interest Rate Year)")
  plt.xlabel("Date")
  plt.ylabel("Adjusted Close Price")
  plt.legend()
  plt.show()

  ####THIS IS THE SAME AS THE CODE ABOVE BUT CHANGED THE YEAR FOR A CLOSER LOOK

  # Fetch S&P 500 data using yfinance
  ticker = "^GSPC"
  start_date = "1980-01-01"
  end_date = "1990-01-01"
  spx_data = yf.download(ticker, start=start_date, end=end_date)

  # Fetch historical interest rates data using the FRED API
  fred = Fred(api_key='767f7894f66f59b2796e04a5525ad91e')
  historical_interest_rates_data = fred.get_series("DGS10")

  # Find the years with the highest interest rates
  highest_rate_year = historical_interest_rates_data.idxmax().year

  # Find the corresponding S&P 500 data for the year with the highest interest rate
  spx_data_highest_rate_year = spx_data[spx_data.index.year == highest_rate_year]

  # Plot the S&P 500's historical performance during the year with the highest interest rate using matplotlib
  plt.figure(figsize=(10, 6))
  spx_data["Adj Close"].plot(label="S&P 500")
  spx_data_highest_rate_year["Adj Close"].plot(marker="o", linestyle="", markersize=8, color="red", label="Highest Rate Year")
  plt.title(f"S&P 500 Performance during {highest_rate_year} (Highest Interest Rate Year)")
  plt.xlabel("Date")
  plt.ylabel("Adjusted Close Price")
  plt.legend()
  plt.show()


def brady_q2() :
  import yfinance as yf
  import pandas as pd
  import matplotlib.pyplot as plt

  # Function to calculate the start date of each president's term
  def calculate_start_dates():
      start_dates = pd.to_datetime([
          '1974-08-09', '1977-01-20', '1981-01-20', '1985-01-21', '1989-01-20', '1993-01-20', '1997-01-20',
          '2001-01-20', '2005-01-20', '2009-01-20', '2013-01-21', '2017-01-20', '2021-01-20'
      ])
      return start_dates

  # Function to plot the performance during the first 100 days of each president's term
  def plot_first_100_days_performance(data):
      first_days = calculate_start_dates()
      plt.figure(figsize=(12, 8))
      for i, first_day in enumerate(first_days):
          closest_date_index = data.index.get_indexer([first_day])[0]
          closest_date = data.index[closest_date_index]
          start_index = data.index.get_loc(closest_date)
          end_index = start_index + 100
          president_data = data.iloc[start_index:end_index]

          # Choose a color for each president
          colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'tab:orange', 'tab:brown', 'tab:gray', 'tab:purple', 'tab:pink', 'tab:olive']
          plt.plot(president_data.index, president_data['Close'], color=colors[i], label=f"President {i + 1}")

      plt.legend()
      plt.xlabel("Date")
      plt.ylabel("S&P 500 Close Price")
      plt.title("S&P 500 Performance during First 100 Days of Each President's Term")
      plt.grid(True)  # Add gridlines to the plot
      plt.show()

  # Download S&P 500 data
  spx_data = yf.download('^GSPC', start='1972-01-01', end='2023-01-01', progress=False)

  # Call the function to plot the performance during the first 100 days of each president's term
  plot_first_100_days_performance(spx_data)


def brady_q3() :
  import pandas as pd
  import matplotlib.pyplot as plt
  import requests
  from bs4 import BeautifulSoup
  import json

  # Read the CSV file into a pandas DataFrame
  df = pd.read_csv('sp500_index_data.csv')

  # Display the first few rows of the DataFrame
  df.isnull().any()
  df = df.dropna()
  df.isnull().any()
  df.shape
  df.drop_duplicates(inplace=True)
  df.shape
  df.columns

  #Rename the columns
  df.rename(columns={
      'Date':'date',
      'Dividend': 'dividend',
      'Earnings' :'earnings',
      'Consumer Price Index': 'consumer_price_index',
      'Long Interest Rate': 'long_interest_rate',
      'Real Price': 'real_price',
      'Real Dividend': 'real_dividend',
      'Real Earnings': 'real_earnings'
  }, inplace = True)

  df.columns

 # Convert DataFrame to JSON
  json_data = df.to_json(orient='records')


  # Read the CSV file into a pandas DataFrame
  df = pd.read_csv('sp500_index_data.csv')

  # Convert 'Date' column to datetime format
  df['Date'] = pd.to_datetime(df['Date'])

  # Set 'Date' column as the index
  df.set_index('Date', inplace=True)

  # Calculate the yearly average of 'SP500' column
  yearly_average = df['SP500'].resample('Y').mean()



  # URL of the Wikipedia page
  url = "https://en.wikipedia.org/wiki/Economy_of_the_United_States#Data"

  table_class = 'wikitable sortable jquery-tablesorter'
  # Send an HTTP GET request to the URL and get the content
  response = requests.get(url)
  html_content = response.content

  # Parse the HTML content using BeautifulSoup
  # parse data from the html into a beautifulsoup object
  soup = BeautifulSoup(response.text, 'html.parser')
  indiatable=soup.find('table',{'class':"wikitable"})
  #print(indiatable)

  df=pd.read_html(str(indiatable))
  # convert list to dataframe
  df=pd.DataFrame(df[0])

  df.to_csv('wiki_gdp_data.csv', sep=',', index=False, encoding='utf-8')

  #clean the data for easy viewing
  json_string = df.to_json('output.json', orient='records', lines = False, indent =4)

    # Read the CSV file into a pandas DataFrame
  df = pd.read_csv('sp500_index_data.csv')

  # Convert 'Date' column to datetime format
  df['Date'] = pd.to_datetime(df['Date'])

  # Set 'Date' column as the index
  df.set_index('Date', inplace=True)

  # Calculate the yearly average of 'SP500' column
  yearly_average = df['SP500'].resample('Y').mean()

  # Convert the Python dictionary to a pandas DataFrame
  gdp_df = pd.read_json('output.json')

  # Merge the GDP DataFrame with the S&P 500 DataFrame based on the 'Year' column
  merged_df = pd.merge(gdp_df, df, left_on='Year', right_on=df.index.year)

  # Calculate the correlation between the 'GDP (in Bil. US$PPP)' and 'SP500' columns
  correlation = merged_df['GDP (in Bil. US$PPP)'].corr(merged_df['SP500'])

  # Create a scatter plot to visualize the correlation
  plt.figure(figsize=(8, 6))
  plt.scatter(merged_df['GDP (in Bil. US$PPP)'], merged_df['SP500'])
  plt.xlabel('GDP (in Bil. US$PPP)')
  plt.ylabel('S&P 500')
  plt.title(f'Correlation between GDP and S&P 500 (Correlation: {correlation:.2f})')
  plt.grid(True)
  plt.show()


def lizette_q1() :
  import matplotlib.pyplot as plt
  import json
  from wordcloud import WordCloud
  from bs4 import BeautifulSoup
  from newspaper import Article
  import pandas as pd


  df = pd.read_csv('spx.csv')
  df['date'] = pd.to_datetime(df['date'])

  start_date = pd.to_datetime('2011-09-01')
  end_date = pd.to_datetime('2011-10-31')
  protest = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

  start_date_jan = pd.to_datetime('2011-01-01')
  end_date_aug = pd.to_datetime('2011-08-31')
  preProtest = df[(df['date'] >= start_date_jan) & (df['date'] <= end_date_aug)]

  plt.figure(figsize=(10, 6))

  plt.plot(protest['date'], protest['close'], label='During Occupy Wall Street Movement', color='blue')
  plt.plot(preProtest['date'], preProtest['close'], label='Jan-Aug 2011', color='red')

  plt.xlabel('Date')
  plt.ylabel('S&P 500 Index')
  plt.title('''S&P 500 Index Comparison: Prior to and During the Time of the 'Occupy Wall Street Movement' ''')
  plt.legend()
  plt.show()

  ##word cloud
  with open("occupyws.json", 'r') as f:
    news = json.load(f)

  occupy_news = [article for article in news]

  def scrapeArticle(url):
      article = Article(url)
      article.download()
      article.parse()
      return article.text

  text = ''
  for article in occupy_news:
      article_text = scrapeArticle(article['link'])
      text += article_text


  wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

  plt.figure(figsize=(10, 6))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  plt.title("Word Cloud of Articles about Occupy Wall Street Movement")
  plt.show()



def lizette_q2() :
  import pandas as pd
  import matplotlib.pyplot as plt
  data = pd.read_csv('spx.csv')
  data['date'] = pd.to_datetime(data['date'])
  data.set_index('date', inplace=True)


  recent_data = data.loc['2017-09-29':'2018-09-29']
  historical_data = data.loc[:'2017-09-29']


  past_year_returns = recent_data['close'].pct_change()
  historical_returns = historical_data['close'].pct_change()


  plt.figure(figsize=(10, 6))
  plt.plot(past_year_returns.index, past_year_returns, label='Past Year Returns')
  plt.plot(historical_returns.index, historical_returns, label='Historical Returns')
  plt.xlabel('Date')
  plt.ylabel('Returns')
  plt.title('S&P 500 Index Returns Comparison')
  plt.legend()
  plt.show()




def lizette_q3() :
  import pandas as pd
  import matplotlib.pyplot as plt

  ##calculate yearly avg. close data from spf 500
  sp500_data = pd.read_csv('spx.csv')
  sp500_data['date'] = pd.to_datetime(sp500_data['date'])
  sp500_data.set_index('date', inplace=True)
  yearly_avg = sp500_data['close'].resample('Y').mean()


  # Plot the yearly average close
  plt.figure(figsize=(10, 6))
  plt.plot(yearly_avg, marker='o', linestyle='-')
  plt.title('S&P 500 and Unemployment Rate from 1986-2018')
  plt.xlabel('Year')
  plt.ylabel('Average Close')
  plt.grid(True)
  plt.tight_layout()

  # Show the plot
  plt.show()

  ##get unemploy. data
  unemployment_data = pd.read_csv('USUnemploymentRate.csv')
  years = unemployment_data['Year']
  unemployment_rates = unemployment_data['AVG']

  plt.figure(figsize=(10, 6))
  plt.plot(years, unemployment_rates, marker='o', linestyle='-', color='orange')

  plt.title('Yearly Average Unemployment Rates in the US (1986-2018)')
  plt.xlabel('Year')
  plt.ylabel('Unemployment Rate (%)')

  plt.grid(True)
  plt.tight_layout()
  plt.show()


def thieu_q1() :
  import yfinance as yf
  import matplotlib.pyplot as plt
  import os
  import pandas as pd
  import numpy as np
  import requests
  import plotly.graph_objects as go
  from plotly.subplots import make_subplots
  from bs4 import BeautifulSoup
  sp500_symbol = '^GSPC'

  sp500_data = yf.Ticker(sp500_symbol).history(period='max')

  sp500_data['Year'] = sp500_data.index.year
  companies_per_year = sp500_data.groupby('Year').size()

  plt.figure(figsize=(10, 6))
  plt.plot(companies_per_year.index, companies_per_year.values, marker='o', linestyle='-')
  plt.title("Number of Companies Listed on the S&P 500 Over the Years")
  plt.xlabel("Year")
  plt.ylabel("Number of Companies")
  plt.grid(True)
  plt.show()

  plt.plot(sp500_data['Close'], label='S&P 500')
  plt.title('S&P 500 Index')
  plt.xlabel('Date')
  plt.ylabel('Price')
  plt.legend()
  plt.grid(True)
  plt.show()



def thieu_q2() :
  import yfinance as yf
  import matplotlib.pyplot as plt
  import os
  import pandas as pd
  import numpy as np
  import requests
  import plotly.graph_objects as go
  from plotly.subplots import make_subplots
  from bs4 import BeautifulSoup

  sp500_symbol = '^GSPC'
  gold_symbol = 'GC=F'

  sp500_data = yf.Ticker(sp500_symbol).history(period='10y')
  gold_data = yf.Ticker(gold_symbol).history(period='10y')

  plt.figure(figsize=(10, 6))
  plt.plot(sp500_data['Close'], label='S&P 500')
  plt.plot(gold_data['Close'], label='Gold')
  plt.title('S&P 500 Index vs. Gold Price Comparison Over the Years')
  plt.xlabel('Date')
  plt.ylabel('Price')
  plt.legend()
  plt.grid(True)
  plt.show()

  dir0='coinDataSets'
  files=os.listdir(dir0)
  paths=[]
  for item in files:
      paths+=[os.path.join(dir0,item)]
      files2=files.copy()

  rates=[]
  for i in range(len(files)):
      data=pd.read_csv(paths[i])
      data['MA60']=data['Close'].rolling(window=60).mean()
      data['MA60 shift year']=data['MA60'].shift(250)
      data['Growth Rate']=(data['MA60']-data['MA60 shift year'])*100/data['MA60 shift year']
      files[i]=data
      rates+=[[i,files2[i][5:-4],data['Growth Rate'].tolist()[-1]]]

  dataGR=pd.DataFrame(columns=['i','Name','Growth Rate %'],data=rates)
  dataGR=dataGR.dropna()
  dataGR['log growth rate']=dataGR['Growth Rate %'].apply(lambda x:np.log(x))
  dataGR2=dataGR.sort_values('Growth Rate %',ascending=False).reset_index(drop=True)
  display(dataGR2)

  #Plotly format
  name0='Bitcoin'
  i=dataGR2[dataGR2['Name']==name0]['i'].tolist()[0]
  data=files[i]
  fig=make_subplots(specs=[[{"secondary_y":False}]])
  fig.add_trace(go.Scatter(x=data['Date'],y=data['Growth Rate'],name='Growth Rate'),secondary_y=False,)
  fig.update_layout(autosize=False,width=800,height=400,title_text=name0+' Growth Rate')
  fig.update_xaxes(title_text="Date")
  fig.update_yaxes(title_text="Growth Rate %",secondary_y=False)
  fig.show()



def thieu_q3() :
  import yfinance as yf
  import matplotlib.pyplot as plt
  import os
  import pandas as pd
  import numpy as np
  import requests
  import plotly.graph_objects as go
  from plotly.subplots import make_subplots
  from bs4 import BeautifulSoup
  sp500 = yf.Ticker('^GSPC')
  sp500_data = sp500.history(period='10y')

  sp500_data['Daily_Return'] = sp500_data['Close'].pct_change()

  biggest_losses = sp500_data.nsmallest(20, 'Daily_Return')

  plt.figure(figsize=(10, 6))

  plt.plot(sp500_data['Close'], label='S&P 500')

  plt.plot(biggest_losses.index, biggest_losses['Close'], 'ro', label='Biggest Losses')

  plt.title('S&P 500 Index Over the Last 10 Years with Biggest Losses')
  plt.xlabel('Date')
  plt.ylabel('Price')
  plt.legend()
  plt.grid(True)
  plt.show()


  sp500 = yf.Ticker('^GSPC')
  historical_data = sp500.history(period='max')

  historical_data['Daily_Return'] = historical_data['Close'].pct_change()

  biggest_losses = historical_data.nsmallest(10, 'Daily_Return')

  plt.figure(figsize=(10, 6))
  plt.barh(biggest_losses.index.strftime('%Y-%m-%d'), biggest_losses['Daily_Return'], color='red')
  plt.xlabel('Daily Percentage Change')
  plt.ylabel('Date')
  plt.title('Biggest Value Losses in S&P 500')
  plt.gca().invert_yaxis()
  plt.show()



#MENU

repeat = True
while repeat == True:
    print('\t---Pythons That Byte Menu---\n'
    "1) How do closing prices correlate with the S&P 500?\n"
    "2) How does the S&P 500 index affect market phases?\n"
    "3) How do the market phases correlate withe the CBOE volatility index?\n"
    "4) What is the correlation between the CPI and S&P 500?\n"
    "5) What is the relationship between the S&P 500 price action and GNP?\n"
    "6) What is the relationship between US Crude Oil FPP and S&P 500?\n"
    "7) Do presidential administration changes drastically affect the S&P 500?\n"
    "8) Historically, what years have the worst yearly change percentage?\n"
    "9) What years were the best and the lowest in terms of the S&P 500’s value?\n"
    "10) Historically, what year had the highest interests rates, and how did that affect the S&P 500?\n"
    "11) Trends in the S&P 500’s performance during the first 100 days of a new presidential administration?\n"
    "12) Is there any correlation between the S&P 500 and GDP? If there is, is it negative or positive?\n"
    "13) How is the S&P 500 affected during times of protest such as the Occupy Wall Street Movement?\n"
    "14) How has the S&P 500 performed in recents times in comparison to how it’s performed historically?\n"
    "15) What relationship is there with unemployment rates in the US and the S&P 500 index?\n"
    "16) Has there been a significant increase in the number of companies listed on the S&P 500 over the years?\n"
    "17) What seem to be relevant to the S&P 500 index prices?\n"
    "18) Do crashes lead to significant changes in S&P prices?\n"
    "0) Exit")

    choice = input('Enter choice: ')

    if choice == "1" :
      nick_q1()
      continue
    elif choice == "2" :
      nick_q2()
      continue
    elif choice == "3" :
      nick_q3()
      continue
    elif choice == "4" :
      noah_q1()
      continue
    elif choice == "5" :
      noah_q2()
      continue
    elif choice == "6" :
      noah_q3()
      continue
    elif choice == "7" :
      siya_q1()
      continue
    elif choice == "8" :
      siya_q2()
      continue
    elif choice == "9" :
      siya_q3()
      continue
    elif choice == "10" :
      brady_q1()
      continue
    elif choice == "11" :
      brady_q2()
      continue
    elif choice == "12" :
      brady_q3()
      continue
    elif choice == "13" :
      lizette_q1()
      continue
    elif choice == "14" :
      lizette_q2()
      continue
    elif choice == "15" :
      lizette_q3()
      continue
    elif choice == "16" :
      thieu_q1()
      continue
    elif choice == "17" :
      thieu_q2()
      continue
    elif choice == "18" :
      thieu_q3()
      continue
    elif choice == "0" :
      print('Thank you!')
      repeat = False
    else:
      print('Improper input!\n')





	---Pythons That Byte Menu---
1) How do closing prices correlate with the S&P 500?
2) How does the S&P 500 index affect market phases?
3) How do the market phases correlate withe the CBOE volatility index?
4) What is the correlation between the CPI and S&P 500?
5) What is the relationship between the S&P 500 price action and GNP?
6) What is the relationship between US Crude Oil FPP and S&P 500?
7) Do presidential administration changes drastically affect the S&P 500?
8) Historically, what years have the worst yearly change percentage?
9) What years were the best and the lowest in terms of the S&P 500’s value?
10) Historically, what year had the highest interests rates, and how did that affect the S&P 500?
11) Trends in the S&P 500’s performance during the first 100 days of a new presidential administration?
12) Is there any correlation between the S&P 500 and GDP? If there is, is it negative or positive?
13) How is the S&P 500 affected during times of protest such as the Occupy Wall St